In [10]:
import requests
import json
import pandas as pd
import numpy as np
from config import geoapify_key
from py_scripts.Master_df import Master_NC_Dataframe
print(Master_NC_Dataframe.columns)

Index(['County', 'year', 'Active Dentists', 'Active Primary Care Physicians',
       'Beds in General Hospitals', 'Deaths by Injury or Violence', 'Divorces',
       'General Hospital Discharges', 'Infant Deaths',
       'Low-Weight Births Under 2500 Grams',
       ...
       'Percent Unemployed', 'People (Unemployed)',
       'National Rank (Unemployment)', 'Black Population',
       'Population (Census/Estimate/Projection)',
       'Population Density (Persons per Square Mile)', 'county longitude',
       'county latitude', 'place_id', 'death_per_1000_infant_death'],
      dtype='object', length=204)


In [16]:
# List of place IDs to iterate over
place_ids = Master_NC_Dataframe["place_id"].tolist()

geoapify_places_url = "https://api.geoapify.com/v2/places?"
# List to store the JSON responses
i = 0
responses = []
for place_id in place_ids:
    # Parameters for the API request
    params = {
        "categories": "healthcare.hospital",
        "filter": f"place:{place_id}",
        "limit": "500",
        "apiKey": geoapify_key  # replace with actual API key
    }
    # Make the request
    response = requests.get(geoapify_places_url, params=params)
    
    if response.status_code == 200:
        # Append the JSON response to the list
        responses.append(response.json())
        i = i + 1
        print(f"Processing record {i} of {len(place_ids)}")
    else: 
        print(f"Error: {response.status_code}")
        break

# Save the list of responses to a JSON file
with open('Resources/geoapify_county_hospitals.json', 'w') as f:
    json.dump(responses, f)
    

Processing record 1 of 100
Processing record 2 of 100
Processing record 3 of 100
Processing record 4 of 100
Processing record 5 of 100
Processing record 6 of 100
Processing record 7 of 100
Processing record 8 of 100
Processing record 9 of 100
Processing record 10 of 100
Processing record 11 of 100
Processing record 12 of 100
Processing record 13 of 100
Processing record 14 of 100
Processing record 15 of 100
Processing record 16 of 100
Processing record 17 of 100
Processing record 18 of 100
Processing record 19 of 100
Processing record 20 of 100
Processing record 21 of 100
Processing record 22 of 100
Processing record 23 of 100
Processing record 24 of 100
Processing record 25 of 100
Processing record 26 of 100
Processing record 27 of 100
Processing record 28 of 100
Processing record 29 of 100
Processing record 30 of 100
Processing record 31 of 100
Processing record 32 of 100
Processing record 33 of 100
Processing record 34 of 100
Processing record 35 of 100
Processing record 36 of 100
P

In [31]:
#create a dataframe with the data from the API call
#normailze the data
#json dump responses
# Load the json file
with open("Resources/geoapify_county_hospitals.json") as f:
    data = json.load(f)

# Define the record path
record_path = ['features', 'properties']

# Extract the nested data into a DataFrame
df_list = []
# For each 'FeatureCollection' in your data
for feature_collection in data:
    # Check if there are any 'features' in the 'FeatureCollection'
    if feature_collection['features']:
        # For each 'feature' in 'features', normalize the 'properties' dictionary and append it to df_list
        for feature in feature_collection['features']:
            df_list.append(pd.json_normalize(feature['properties']))

# Concatenate all the dataframes in the list into a single dataframe
hospitals_geoapify_df = pd.concat(df_list, ignore_index=True)



columns = [
    'county',
    'city',
    'name',
    'postcode',
    'lon',
    'lat',
    'formatted',
    'categories',
    'place_id',
    'datasource.raw.name',
    'datasource.raw.phone',
    'datasource.raw.website'
    ]
hospitals_geoapify_df = hospitals_geoapify_df[columns]
hospitals_geoapify_df.rename(columns={'county': 'County', 'city': 'Hospital City', 'name': 'Hospital Name', 'postcode': 'Hospital Zipcode', 'formatted': 'Hospital Address', 'categories': 'Hospital Geoapify Category', 'place_id': 'Hospital Place ID', 'datasource.raw.name': 'Source Hospital Name', 'datasource.raw.phone': 'Hospital Phone', 'datasource.raw.website': 'Hospital Website'}, inplace=True)
hospitals_geoapify_df['Hospital Name'].fillna('No Name Given - Possible branch of main Hospital', inplace=True)
print(hospitals_geoapify_df.head())
hospitals_geoapify_df.to_csv("Clean_Resources/hospitals_geoapify.csv", index=False)

             County Hospital City  \
0   Alamance County    Burlington   
1  Alleghany County        Sparta   
2      Anson County     Wadesboro   
3       Ashe County     Jefferson   
4      Avery County           NaN   

                                      Hospital Name Hospital Zipcode  \
0                  Alamance Regional Medical Center            27215   
1  No Name Given - Possible branch of main Hospital            28675   
2                          Anson Community Hospital            28170   
3                            Ashe Memorial Hospital            28640   
4  No Name Given - Possible branch of main Hospital            28653   

         lon        lat                                   Hospital Address  \
0 -79.501377  36.061832  Alamance Regional Medical Center, 1240 Huffman...   
1 -81.114737  36.509522  Cherry Street, Sparta, NC 28675, United States...   
2 -80.110058  34.977514  Anson Community Hospital, 2301 W. Cliff Martin...   
3 -81.457822  36.423469  Ashe Me

In [34]:
!jupyter nbconvert --to script api_call.ipynb --output ./py_scripts/Hospitals_df

[NbConvertApp] Converting notebook api_call.ipynb to script
[NbConvertApp] Writing 3146 bytes to py_scripts\Hospitals_df.py
